# Weekly Rate of Return, Sharpe, Jensen, Betas, and Mean Variance Weights

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from tabulate import tabulate
import math
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")

# yahoo_finance is used to fetch data 
import yfinance as yf

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

one_yr_ago = datetime.now() - relativedelta(years=1)
one_yr_ago = one_yr_ago.strftime("%Y-%m-%d")

start = one_yr_ago 
end = '2023-07-31'
symbols = ['FDS','INVH', 'ROL', 'RBLX', 'HIG', 'SNOW']
all_df = yf.download(symbols, start, end)['Adj Close']

[*********************100%***********************]  6 of 6 completed


In [3]:
all_df = yf.download(symbols, start, end)['Adj Close']

[*********************100%***********************]  6 of 6 completed


In [4]:
df = all_df[symbols]
df = df.reset_index() 

In [5]:
df

,Date,FDS,INVH,ROL,RBLX,HIG,SNOW
0,2022-08-05,419.673248,36.814873,37.131447,49.240002,62.684643,165.529999
1,2022-08-08,419.911163,37.069038,37.309208,48.900002,63.114517,170.850006
2,2022-08-09,420.109467,37.587147,36.675507,47.349998,64.247841,160.550003
3,2022-08-10,431.163666,38.066147,36.962658,48.009998,65.283470,174.399994
4,2022-08-11,429.428711,38.017269,36.289349,49.560001,66.104164,167.880005
...,...,...,...,...,...,...,...
241,2023-07-24,436.739990,36.160000,44.750000,39.549999,74.839996,174.800003
242,2023-07-25,439.239990,35.549999,44.830002,39.660000,74.510002,179.289993
243,2023-07-26,440.420013,35.520000,44.799999,39.950001,75.660004,170.000000
244,2023-07-27,435.440002,34.889999,40.820000,37.910000,76.070000,168.270004


# Standard Deviation (weekly)

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fmin
import math


In [7]:
def std_dev_portfolio(weekly_returns, W):
    return np.dot(W,  np.array(np.std(weekly_returns, axis=0)))

In [8]:
def sharpe_ratio(daily_returns, W,risk_free_rate=0.0):
    # Calculate aggregated Sharpe Ratio
    daily_sharpe_ratio = (np.dot(W,  np.array(np.mean(daily_returns, axis=0))) - risk_free_rate) 
    daily_sharpe_ratio = daily_sharpe_ratio/np.dot(W,  np.array(np.std(daily_returns, axis=0)))
    sharpe_ratio = daily_sharpe_ratio 
    return sharpe_ratio

# Sharpe Ratio opt_weights

In [9]:
print('Symbols: ', symbols)

opt_weights = [3.44608226e-01, 6.12683089e-02, 8.42134112e-02, 2.97174507e-08,
       3.81387918e-01, 1.28522106e-01]
print("with weights: ", opt_weights) 
#tot_returns = df[symbols].pct_change(len(df)-1)
tot_return = df[symbols]*opt_weights
tot_return = tot_return.pct_change(len(df)-1).iloc[-1].sum()
tot_return = tot_return*100 
std_dev_port = std_dev_portfolio(df[symbols], opt_weights)
sharpe = (tot_return / (std_dev_port))
print('Portfolio Sharpe ratio: ',round(sharpe, 4))

print('Standard deviation: ',str(round(std_dev_port, 4))+"%")

Symbols:  ['FDS', 'INVH', 'ROL', 'RBLX', 'HIG', 'SNOW']
with weights:  [0.344608226, 0.0612683089, 0.0842134112, 2.97174507e-08, 0.381387918, 0.128522106]
Portfolio Sharpe ratio:  0.9078
Standard deviation:  10.6904%


# Sharpe Ratio equal weights

In [10]:
equal_weighted = [1/len(symbols) for i in range(len(symbols))]
tot_return = df[symbols]*equal_weighted
tot_return = tot_return.pct_change(len(df)-1).iloc[-1].sum()
tot_return = tot_return*100 
std_dev_port = std_dev_portfolio(df[symbols], equal_weighted)
sharpe = (tot_return / (std_dev_port))
print('Portfolio Sharpe ratio: ',round(sharpe, 4))

print('Standard deviation: ',str(round(std_dev_port, 4))+"%")

Portfolio Sharpe ratio:  1.149
Standard deviation:  8.4461%


# Jensen's Index


In [11]:
def jensen_indx(tot_return, market_return, beta, risk_free_rate=0.0):
    return (tot_return - (beta * (market_return-risk_free_rate)))

In [12]:
opt_weights = [3.44608226e-01, 6.12683089e-02, 8.42134112e-02, 2.97174507e-08,
       3.81387918e-01, 1.28522106e-01]

In [13]:
stock_returns = df[symbols].pct_change(len(df)-1).iloc[-1]* opt_weights
stock_returns = stock_returns.sum()

In [14]:
df = all_df[symbols]
df = df.reset_index()

In [15]:
tot_return = df[symbols]*opt_weights
tot_return = tot_return.pct_change(len(df)-1).iloc[-1].sum()
tot_return = tot_return*100
# market S&P return over 5 yrs
#  from: https://www.fool.com/investing/how-to-invest/index-funds/average-return/
market_return= 10.7  

In [16]:
tot_return, market_return,

(9.704476565506969, 10.7)

In [17]:
beta = 0.9204 # from previous section

j_ind= jensen_indx(tot_return, market_return, beta)
print("Jensen's Index: ",str(round(j_ind, 4)))

Jensen's Index:  -0.1438
